## Phylogenetics in R

### 1. Introduction and resources

This practical introduces you to basic phylogenetic computing in `R`. We will review importing phylogenetic trees as data files, displaying phylogenetic trees visually, and some basic evolutionary computations that can be conducted with phylogenetic trees. This practical will deliver some of the important background for Coursework 1. Below you will find some of the relevant resources required for this practical.

Parts (sections 2,3,4) of this practical are written by [Natalie Cooper](http://nhcooper123.github.io/).
The original can be found [here](https://github.com/nhcooper123/TeachingMaterials/blob/master/PhD_Museum/VisualisingPhylo.Rmd). Whereas, parts (section 5 & 6) of this practical are written by Adam Devenish & Rob Barber (a.devenish@imperial.ac.uk) (r.barber19@imperial.ac.uk).

The data used throughout the practical can be downloaded from Blackboard, Binder, or from [here](https://raw.githubusercontent.com/nhcooper123/TeachingMaterials/master/PhD_Museum/elopomorph.tre) and [here](https://raw.githubusercontent.com/nhcooper123/TeachingMaterials/master/PhD_Museum/anole.data.csv) (right click and `save link as`).



### 2. Installing and loading extra packages in R
To plot phylogenies (or use any specialized analysis) in R, you need to download one or more additional
packages from the basic R installation. For this practical you will need to install the following packages:

• ape

• phytools

To install the package ape:

In [ ]:
dir.create("My Git Repo")
git2r::clone("https://github.com/Syrph/BCB_Practicals", "My Git Repo")
setwd("My Git Repo")
system("sudo apt-get install libgdal-dev libproj-dev libgeos-dev libudunits2-dev libv8-dev libprotobuf-dev libjq-dev")
source("install.R")

In [ ]:
install.packages("ape")

Pick the closest mirror to you. These packages will download on both Jupyter Notebooks and in your own RStudio.

You've installed the packages but they don't automatically get loaded into your `R` session. 
Instead you need to tell `R` to load them **every time** you start a new `R` session and want to use functions from these packages. 
To load the package `ape` into your current `R` session:

In [ ]:
library(ape)

You can think of `install.packages` like installing an app from the App Store on your smart phone - you only do this once - and `library` as being like pushing the app button on your phone - you do this every time you want to use the app.

Don't forget to load `phytools` too! This may take a while on Juypter Notebooks, but you only need to install it once on your own computer!

In [ ]:
install.packages("phytools",repos="https://cloud.r-project.org",quiet=TRUE)

In [ ]:
library(phytools)

### 2. A refresher of phylogenetic trees

This section will review some basic aspects of phylogenetic trees and introduce how trees are handled at the level of software. Because you are now interacting with phylogenetic trees at a ‘lower level’ (i.e. at the bioinformatics level), it is also helpful to know some of the names for parts of phylogenetic trees used in computer science.

### A. Tree parameters
 
A phylogenetic tree is an ordered, multifurcating graph with labeled **tips** (or **leaves**) (and sometimes labeled histories). It represents the relative degrees of relationships of species (i.e. tips or OTUs). The graph consists of a series of **branches** (or **edges**) with join successively towards **nodes** (or **vertices**, *sing.* **vertex**). Each node is subtended by a single branch, representing the lineage of ancestors leading to a node. The node is thus the common ancestor of two or more descendant branches. All the descendant branches of a given node (and all of the their respective descendants) are said to form a **clade** (or **monophyletic group**).

In [ ]:
# rtree creates a random tree. Setting the seed means that the same 'random' tree will plotted each time. This is because random
# numbers from your computer are generated using the internal clock. Google it for more info!
set.seed(0); plot(rtree(10), "unrooted") 
## Plot clade label
rect(1.3,2.0,2.5,2.7, border = "grey")
text(2.8, 2.5, "Clade")
## Plot node label
arrows(0.35,0.9,0.75,1, length = 0.125, angle = 20, code = 1)
text(1.0, 1, "Node")
## Plot edge label
arrows(0.65,1.45,0.15,1.4, length = 0.125, angle = 20, code = 1)
text(-0.1, 1.4, "Edge")
## Plot tip label
arrows(-0.15,2.8,-0.45,2.95, length = 0.125, angle = 20, code = 1)
text(-0.6, 3, "Tip")

When we select a node to act as the base of a tree, the tree is said to be **rooted**. At the bottom of a tree, is the **root node** (or simply the **root**).

In [ ]:
# You can also read in trees directly from text by specifying the branch lengths after each node/tip. More info below!
tree <- read.tree(text = "(((Homo:1, Pan:1):1, Gorilla:1):1, Pongo:1);")
plot(tree)
## Plot root label
lines(c(-0.5,0), c(3.18,3.18))
arrows(0.03,3.18,0.35,3.18, length = 0.125, angle = 20, code = 1)
text(0.5, 3.18, "Root")

Phylogenetic trees of the kind shown above are fairly simple and lack information about time or character changes occurring along a branch. We can assign branch length in the form of either time or the amount of change/substitution along a branch. A tree with **branch lengths** depicted can be called a **phylogram**.

When (an implied) dimension of time is being considered, all the tips of the tree must be at the level representing the time in which they are observed. For trees where all the species are extant, the tips are flush at the top. This representation is called an **ultrametric** tree.

In [ ]:
tree <- read.tree(text = "(((Homo:6.3, Pan:6.3):2.5, Gorilla:8.8):6.9, Pongo:15.7);")
plot(tree)
axisPhylo()

### B. Informatic representations of tree

To perform any useful calculations on a tree, we need both a computer-readable tree format and (in part) to understand how trees are constructed in computer memory.
 
#### Text based formats

Storage of trees for transfer between different software is essential. This is most commonly achieved with a text-based format stored in a file. The most common file format for representing phylogenetic trees is **Newick format**. This consists of clades represented within parentheses. Commas separate each clade. Either tip names or symbols representing the tips are nested within the lowest orders of parentheses. Each tip or branch can be associated with a branch length scalar that follows a colon.

For example:

`"(((Homo, Pan), Gorilla), Pongo);"`

Or with branch length:

`"(((Homo:6.3, Pan:6.3):2.5, Gorilla:8.8):6.9, Pongo:15.7);"`

Trees are also increasing use of XML formats such as PhyloXML and NeXML.

In this practical we are going to use the `elopomorph.tre` newick tree.
You can open it with a simple text editor to see the newick tree structure.

#### Edge table

It is also possible to represent a phylogenetic tree as a matrix of edges and vertices called an edge table. This is an even less intuitive representation, but it is implemented in `R` and worth reviewing here.

There are a number of conventions that can be used to create an edge table. The general concept consists of numbering the tips *1 - n*, and all internal nodes labeled *n+1 ... n+n-1*. The numbers for the internal nodes can be assigned arbitrarily or according to an algorithm.

In `R` packages like `ape`, edge tables are constructed as follows:

| node | connects to |
|---|---|
| 5 | 6 |
| 6 | 7 |
| 7 | 1 |
| 7 | 2 |
| 6 | 3 |
| 5 | 4 |

You read the table as follows: node `5` (root) connects to node `6`. The node `6` connects to node `7`. Node `7` connects to node `1` that happen to be the first tip (`Homo`) and to node `2` (`Pan`) etc... Note that in a binary tree (i.e. a tree where each node has only two descendants) each node always connects to two elements (nodes or tips).

In [ ]:
tree <- read.tree(text = "(((Homo, Pan), Gorilla), Pongo);")
plot(tree, label.offset = 0.1)
nodelabels() ; tiplabels()

#### Records & pointers

At a lower level, phylogenetic trees can be represented in computer memory as more complex data objects. We don’t need to go into detail here, but if you consider nodes and tips as data objects (i.e. a dataframe), a tree could be stored as an array of dataframes which store information about which store information about which members of that same array are descendants and which are ancestors.

### 3. Loading your phylogeny and data into `R`
#### Reading in a phylogeny from a file
To load a tree you need the function `read.tree`.
`read.tree` can read any newick format trees (see above) like the `elopomorph.tre` file.

In [ ]:
fishtree <- read.tree("elopomorph.tre")

###### Be sure you are always in the right directory. Remember you can navigate in `R` using `setwd()`, `getwd()` and `list.files()` (to see what's in the current directory). While using this notebook all the files are stored in the main binder, so you shouldn't need to change the directory.

### Reading in a phylogeny that is already built into `R`
The bird and anole phylogenies are already built into `R` so we don't need to read them in using `read.tree`.
Instead we just use:


In [ ]:
data(bird.orders)
data(anoletree)

### Reading and viewing your data in `R`
Later we will use some Greater Antillean *Anolis* lizard data to add data to a phylogeny.
Before we can add data to our tree, we need to load the data we are going to use. 
`R` can read files in lots of formats, including comma-delimited and tab-delimited files.
Excel (and many other applications) can output files in this format (it's an option in the `Save As` dialogue box under the `File` menu). 
To save time I have given you a comma-delimited text file called `anole.data.csv` which we are going to use. 
Load these data as follows. 
I am assuming you have set your working directory, if not don't forget the path.

In [ ]:
anoledata <- read.csv("anole.data.csv", header = TRUE)

You can use `read.delim` for tab delimited files or `read.csv` for comma delimited files (**c**omma **s**eparated **v**alues).
`header = TRUE`, indicates that the first line of the data contains column headings.

This is a good point to note that unless you **tell** `R` you want to do something, it won't do it automatically. 
So here if you successfully entered the data, `R` won't give you any indication that it worked.
Instead you need to specifically ask `R` to look at the data.

We can look at the data by typing:

In [ ]:
str(anoledata)

**Always** look at your data before beginning any analysis to check it read in correctly.

`str` shows the structure of the data frame (this can be a really useful command when you have a big data file). 
It also tells you what kind of variables `R` thinks you have (characters, integers, numeric, factors etc.). 
Some `R` functions need the data to be certain kinds of variables so it's useful to check this.

In [ ]:
head(anoledata)

This gives you the first few rows of data along with the column headings.

In [ ]:
names(anoledata)

This gives you the names of the columns.

In [ ]:
anoledata

This will print out all of the data!

### 4. Basic tree viewing in `R`
Now let's visualise some phylogenies! We'll use the Elopomorpha (eels and similar fishes) tree to start as it is simple.

In [ ]:
fishtree <- read.tree("elopomorph.tre")

Let's examine the tree by typing:

In [ ]:
fishtree
str(fishtree)

`fishtree` is a fully resolved tree with branch lengths. 
There are 62 species and 61 internal nodes. 
We can plot the tree by using the `plot.phylo` function of `ape`. 
Note that we can just use the function `plot` to do this as `R` knows if we ask it to plot a phylogeny to use `plot.phylo` instead!

In [ ]:
plot(fishtree, cex = 0.5)

`cex = 0.5` reduces the size of the tip labels so we can read them. 

We can also zoom into different sections of the tree that you're interested in:

In [ ]:
zoom(fishtree, grep("Gymnothorax", fishtree$tip.label), subtree = FALSE, cex = 0.8)

The `grep` function is a generic function in `R` that allows to *grab* any element in an object containing the desired characters.
In this example, `grep` is going to search for all the elements in `fishtree$tip.label` that contains `Gymnothorax` (e.g. `Gymnothorax_kidako`, `Gymnothorax_reticularis`).
Try using only `grep("thorax", fishtree$tip.label)` to see if it also only selects the members of the *Gymnothorax* genus.

In this example, we just display the tree for the *Gymnothorax* genus but you can also see how the species fit into the rest of the tree using:


In [ ]:
zoom(fishtree, grep("Gymnothorax", fishtree$tip.label), subtree = TRUE, cex = 0.8)

###### Note that `zoom` is a specific plotting function that will automatically set the plotting window to display two plots at once. This might create some conflicts if you're using RStudio. The bug can be easily solved though by typing `dev.off()` to reinitialise the plotting window and then proceed to the normal `zoom(...)` function as written above.

You can also reset this to one plot only per window by using:


In [ ]:
par(mfrow = c(1, 1))

To get further options for the plotting of phylogenies:

In [ ]:
?plot.phylo

Using the question mark (`?`) can also be done for every function if you want more details!

Note that although you can use `plot` to plot the phylogeny, you need to specify `plot.phylo` to find out the options for plotting trees. You can change the style of the tree (`type`), the color of the branches and tips (`edge.color`, `tip.color`), and the size of the tip labels (`cex`). 
Here's an fun/hideous example! 


In [ ]:
plot(fishtree, type = "unrooted", edge.color = "deeppink", tip.color = "springgreen",  cex = 0.5)

Or try

In [ ]:
plot(ladderize(fishtree), type = "c", edge.color = "darkviolet", tip.color = "hotpink",  cex = 0.5)

The `ladderize` function allows to display the branches from shortest to longest.

> Try to modify the graphical options (colors, display, size, ordering of the nodes, etc.) to obtain the most beautiful or ugliest Elopomorpha phylogeny!

## 5. Manipulating phylogenetic trees in `R`

There are a range of ways in which we can manipulate trees in R. To start lets take a look at the bird family Turdidae.

In [ ]:
Turdidae_tree<-read.nexus("Turdidae_birdtree.nex")

As this multiphylo object (i.e. contains 100 different trees) we need to first choose one random tree before we start.

In [ ]:
Ran_Turdidae_tree <-sample(Turdidae_tree,size=1)[[1]]
plotTree(Ran_Turdidae_tree,type="fan",fsize=0.4,lwd=0.5,ftype="i")

First, lets see what species are in the tree.

In [ ]:
Ran_Turdidae_tree$tip.label

Lets say we want to drop all species with the Myadestes genus. In this instance we first find all the associated tip.labels. 

In [ ]:
drop.species<-("Myadestes")
# sapply is a function that will iterate a given function over a vector (check out apply, lapply, mapply for more info)
# in this case, we're using the grep function to find the index of anything in tip.labels that matches drop.species
selected.tips<-sapply(drop.species,grep,Ran_Turdidae_tree$tip.label)
# We then use the indices to select only the tips we want from all the available tip labels
drop.species<-Ran_Turdidae_tree$tip.label[selected.tips]
drop.species

Now, we create a new tree, with the Myadestes tips dropped from it.

In [ ]:
# drop.tip will remove any matching tips from the tree
Ran_Turdidae_tree_NM<-drop.tip(Ran_Turdidae_tree,drop.species)
plotTree(Ran_Turdidae_tree_NM,type="fan",fsize=0.4,lwd=0.5,ftype="i")

Alternatively, lets say we want to extract the clade within the tree that includes the pre identified selected range of species.

In [ ]:
# Set diff will find all the tips that don't match drop.species, and then drop.tip will remove them
pruned_birdtree<-drop.tip(Ran_Turdidae_tree,
    setdiff(Ran_Turdidae_tree$tip.label,drop.species))
plotTree(pruned_birdtree,ftype="i")

**Remember, always to save your new revised tree. As this will save you time from having to prune it everytime. Use `write.tree` or `write.nexus` the same as you would to save a dataframe.**

For some analyses, you might want to work with on a genus level tree. This can easily be done by a few key steps. We can do this using base `R`, but we'll use some packages from the `tidyverse` that have some very useful functions. 
First install and load `stringr` (for manipulating strings) and `dplyr` (for manipulating dataframes). 

In [ ]:
install.packages(c('stringr', 'dplyr'))
library(stringr)
library(dplyr)

In [ ]:
# Copy a list of all the tips from the tree
bird_tips <- Ran_Turdidae_tree$tip.label

# Split the labels into two strings where there's an underscore (simplfy returns the splits as separate columns in a dataframe)
bird_genera <- bird_tips %>% str_split(pattern = "_", simplify= TRUE)
colnames(bird_genera) <- c("Genus", "Species")

# Pull out the rows that have an distinct genus name. This will be the first instance in the dataframe for that genus.
bird_genera <- as.data.frame(bird_genera) %>% distinct(Genus, .keep_all = T)
bird_genera

We now have a dataframe of species, with one tip per each unique genus. 

A few things to note about the 'tidy' code we just ran:

`%>%` is an operator used to 'pipe' an object into a function. Piping is common in most computing languages so look it up for more information! It can make code less clutered by separating the data from the function you need to use. This isn't unique to `tidyverse` functions, but you'll see it used a lot more in their documentation.

For the `distinct` function, we specified a column name without using quotation marks. To make code easier to read, most tidy functions will process column names (or other labels) this way.

We can now use our unique species from each genera to drop all the other tips in the tree.

In [ ]:
# Combine the columns and add back in the underscore so they match the labels in the tree
genera_tips <- paste(bird_genera$Genus, bird_genera$Species, sep="_")

# Pull out the tips we want to drop
drop.tips <- setdiff(Ran_Turdidae_tree$tip.label, genera_tips)

# Remove all the species except one per genus
genera_tree<-drop.tip(Ran_Turdidae_tree, drop.tips)
plotTree(genera_tree,ftype="i")

As its dropped all but one species per genus, this means we will finally need to also rename the tip labels as well to reflect this change.


In [ ]:
# It's definitely worth checking that the labels match up properly when you change tip labels
par(mfrow=c(1,2))
plotTree(genera_tree,ftype="i")

# Swap species names for genera
genera_tree$tip.label<- bird_genera$Genus
plotTree(genera_tree,ftype="i")

It is important to note, that a big limitation with this approach is that by selecting only one species per genus to keep, that you may run the risk of unintentially dropping tips of species that are paraphyletic. For example, Zoothera genus is spread throughout Turdidae tree.

In [ ]:
plotTree(Ran_Turdidae_tree,fsize=0.2,lwd=0.2,ftype="i")
zoom(Ran_Turdidae_tree, grep("Zoothera", Ran_Turdidae_tree$tip.label), subtree = TRUE, cex = 0.8)

This means that when collapsing a phylogenetic tree you run the risk of miss representing the relationship between the different genera. The only way to get round this is by:

a) making sure check to see how paraphyletic your tree is at the start; this can be done more easily by uploading and viewing your tree file @ https://itol.embl.de/. 
b) Renaming your conflicting paraphyletic clades within your phylogeny, by altering the individual species names.


In [ ]:
Ran_Turdidae_tree$tip.label[Ran_Turdidae_tree$tip.label=="Turdus_philomelos"]<-"Turdus1_philomelos"

## 6. Adding trait data to trees in `R`

Often basic tree plots in R are all you need for exploring data and your analysis. However, for publications and presentations it may be useful to plot trees with associated trait data. We will try plotting data with a tree, using the package `ggtree`, and extension of `ggplot2`.

For this exercise we will use the Turdidae tree (Thrushes) with some data on different habitat types.
Load in the data:

In [ ]:
turdidae_data <- read.csv("Turdidae_data.csv")
str(turdidae_data)

We first need to match our data to the tip labels. First we need to put an underscore in the names, and then match them to see if there's any name differences or missing species. In your coursework you will find that occasionally because of taxonomic disagreements, you might be missing species. The easiset way to solve this is by manually checking tips that don't match.

In [ ]:
# Replace the blank space with an underscore
turdidae_data$Jetz_Name <- turdidae_data$Jetz_Name %>% str_replace(" ", "_")
head(turdidae_data)

We'll use the `%in%` operator, which is useful checking if our species are in the tip labels

In [ ]:
turdidae_data$Jetz_Name %in% Ran_Turdidae_tree$tip.label

We can save the results, and use this to select the rows we need from turdidae_data. We'll use the `!` operator, which means NOT. So in this case it's the species that are not in the tip labels. This can also work for lots of other functions as well so try experimenting!

In [ ]:
index <- !(turdidae_data$Jetz_Name %in% Ran_Turdidae_tree$tip.label)
turdidae_data[index,]

One of them we renamed so we'll change that back for our plots. The others aren't in our taxonomy. This is because they've been moved to Chaetopidae, a new family of just rockjumpers. This often happens as there are three main bird taxonomies: Jetz, Birdlife, and American Ornithological Society. We'll just remove them from our dataset.

In [ ]:
# Change the name back
Ran_Turdidae_tree$tip.label[Ran_Turdidae_tree$tip.label=="Turdus1_philomelos"]<-"Turdus_philomelos"
# Get the species that ARE in the tips
index <- turdidae_data$Jetz_Name %in% Ran_Turdidae_tree$tip.label
# Select only these species
turdidae_data <- turdidae_data[index,]

Now we can drop the tips that don't match and get plotting!

In [ ]:
drop.tips <- setdiff(Ran_Turdidae_tree$tip.label, turdidae_data$Jetz_Name)

# Remove all the species except one per genus
turdi_tree <-drop.tip(Ran_Turdidae_tree, drop.tips)
plotTree(turdi_tree,ftype="i")

Now lets try using ggtree to plot our data with our phylogeny. First we need to install `ggtree`. Because not all packages are available from `CRAN` directly through `R`, we'll install `BiocManager`. `devtools` is another great package for installing packages from github.

In [ ]:
install.packages("BiocManager")
BiocManager::install("ggtree")

In [ ]:
library(ggtree)
library(ggplot2)

`ggtree` is a bit more complicated than just normal tree plots, but you can also do a lot more. We'll create a basic tree plot structure first and then add tip labels and traits after. We'll also make the plotting window for `Jupyter Notebook` bigger.

In [ ]:
# This resizes the plot windows in our notebook. In RStudio you can do this when you export plots using jpeg() etc.
options(repr.plot.width=15, repr.plot.height=15)
turdidae_plot <- ggtree(turdi_tree, layout = "circular")
turdidae_plot

Now we'll create a simple one column dataframe with just the habitat data to plot and the species names as row names.

In [ ]:
habitat_data <- as.data.frame(turdidae_data[,2])
rownames(habitat_data) <- turdidae_data$Jetz_Name
colnames(habitat_data) <- "Habitat"

Now we can make our plot!

In [ ]:
# Create the heat map using habitat data. Offset tells us how far from the tips to plot the data and width is how wide the bars are.
gheat <- gheatmap(p=turdidae_plot, data=habitat_data, offset = 0.05, width=0.1, colnames = FALSE) + 
# Scale fill manual is a ggplot function. It lets the plot know what to 'fill' and what colours to use.
scale_fill_manual(name = 'Habitat', values=c("deeppink3", "goldenrod2", "cyan4")) + 
# Add the tip labels and offset them further from the bars
geom_tiplab(offset = 5, cex=4) + 
# We're adding tip labels on top of an existing plot, so we need to make the axes bigger so our labels don't fall off the edge.
xlim(0, 40) +
# This resizes the legend and makes the text bigger than the default. Try changing the numbers!
theme(legend.key.width = unit(0.8, "cm"), legend.key.height = unit(0.8, "cm"), legend.text = element_text(size = 13), legend.title = element_text(size = 13))

# Lastly plot the data!
plot(gheat)

And now we have a plot where we can see the spread of habitat types in Thrushes. Try experimenting with different colours and sizes to create some beautiful trees that put this one to shame! There's also lots of other ways you can label trees. For more info this guide is a great place to start: 
https://4va.github.io/biodatasci/r-ggtree.html#the_ggtree_package